* **검증 세트**는 하이퍼파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트이다.
* **교차 검증**은 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련한다. 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법이다.
* **그리드 서치**는 하이퍼파라미터 탐색을 자동화해주는 도구이다. 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택한다. 마지막으로 이 매개변수 조합으로 최종 모델을 훈련한다.
* **랜덤 서치**는 연속된 매개변수 값을 탐색할 때 유용하다. 탐색할 값을 직접 나열하는 것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달한다. 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있다.

# scikit-learn

* **cross_validate()**는 교차 검증을 수행하는 함수이다.
  * 첫 번째 매개변수에 교창 검증을 수행할 모델 객체를 전달한다. 두 번째와 세 번째 매개변수에 특성과 타깃 데이터를 전달한다.
  * scoring 매개변수에 검증에 사용할 평가 지표를 지정할 수 있다. 기본적으로 분류 모델은 정확도를 의미하는 'accuarcy', 회귀 모델은 결정계수를 의미하는 'r2'가 된다.
  * cv 매개변수에 교차 검증 폴드 수나 스플리터 객체를 지정할 수 있다. 기본값은 5이다. 회귀일 때는 KFold 클래스를 사용하고 분류일 때는 StratifiedKFold 클래스를 사용하여 5-폴드 교차 검증을 수행한다.
  * n_jobs 매개변수는 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정한다. 기본값은 1로 하나의 코어를 사용한다. -1로 지정하면 시스템에 있는 모든 코어를 사용한다.
  * return_train_score 매개변수를 True로 지정함녀 훈련 세트의 점수도 반환한다. 기본값은 False이다.

* **GridSearchCV**는 교차 검증으로 하이퍼파라미터 탐색을 수행한다. 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련한다.
  * 첫 번째  매개변수로 그리드 서치를 수행할 모델 객체를 전달한다. 두 번째 매개변수에는 탐색할 모델의 매개변수와 값을 전달한다.
  * scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate() 함수와 동일하다.
* **RandomizedSearchCV**는 교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행한다. 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련한다.
  * 첫 번째 매개변수로 그리드 서치를 수행할 모델 객체를 전달한다. 두 번째 매개변수에는 탐색할 모델의 매개변수와 확률 분포 객체를 전달한다.
  * scoring, cv, n_jobs, return _train_score 매개변수는 cross_validate()함수와 동일하다.

In [2]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')


data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target ,test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0154891 , 0.01551604, 0.01717782, 0.01956844, 0.01441503]), 'score_time': array([0.00306773, 0.00486493, 0.00287843, 0.00269294, 0.00171661]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [12]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [28]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(5, 20, 1), 'min_samples_split': range(2, 100, 10)}

In [29]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [30]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [32]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([0, 1, 2, 5, 0, 7, 1, 5, 6, 1])

In [33]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([117,  95,  91, 101,  97, 104,  88, 117,  90, 100]))

In [34]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.64202814, 0.80165289, 0.25047741, 0.65403893, 0.35165263,
       0.4354452 , 0.44015888, 0.89605572, 0.56167663, 0.17978975])

In [35]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(20, 50), 'min_samples_split': randint(2, 25), 'min_samples_leaf': randint(1, 25)}

In [37]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs = -1, random_state=42)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ef050ccf550>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ef050ccf100>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ef050cce5c0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ef050cced10>},
                   random_state=42)

In [38]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [41]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
